Let's set up a study workflow using Jupyter Notebooks, LLMs, and langchain.

In [ ]:
# !pip install langchain
# !pip install langchain-openai
# !pip install pypdf
# !pip install chromadb
# !pip install langchainhub

In [3]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

In [7]:
pdf_path = "./assets-resources/Augmenting-Human-Intellect_ A-Conceptual-Framework-1962-Doug Engelbart.pdf"

loader = PyPDFLoader(pdf_path) # LOAD
pdf_docs = loader.load_and_split() # SPLIT
embeddings = OpenAIEmbeddings() # EMBED
vectordb = Chroma.from_documents(pdf_docs, embedding=embeddings) # STORE
retriever = vectordb.as_retriever()
llm = ChatOpenAI(model="gpt-4-1106-preview")
pdf_qa = RetrievalQA.from_llm(llm=llm, retriever=retriever) # RETRIEVE
pdf_qa

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x169b34c10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x169b3fdf0>, model_name='gpt-4-1106-preview', openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy='')), document_prompt=PromptTemplate(input_variables=['page_content'], template='Context:\n{page_content}'), document_variable_name='context'), re

In [9]:
query = "What is augmentation according to Douglas Engelbart?"
result = pdf_qa.invoke({"query": query, "chat_history": []})

In [11]:
print(result["result"])

According to Douglas Engelbart, augmentation refers to the use of computer-based systems to enhance human intellect by facilitating the use of more powerful concepts and concept-manipulation techniques. This involves redesigning the means of intellectual augmentation by leveraging the capabilities of computers to manipulate and portray symbols, which in turn can transform the way individuals think, reason, and solve problems. Engelbart's conceptual framework on augmentation is not just about automating existing processes but about creating new ways to extend human cognitive capabilities through the development of new languages, methodologies, and interactive computer systems.


In [13]:
def ask_pdf(pdf_qa,query):
    print("QUERY: ",query)
    result = pdf_qa.invoke({"query": query, "chat_history": []})
    answer = result["result"]
    print("ANSWER", answer)
    return answer


ask_pdf(pdf_qa,"What are the four ways through which human capabilities are augmented according to Douglas Engelbart?")

QUERY:  What are the four ways through which human capabilities are augmented according to Douglas Engelbart?
ANSWER Douglas Engelbart identified four main ways in which human capabilities are augmented in his conceptual framework:

1. Artifacts: These are the tools and physical instruments that humans create to manipulate the physical environment or to perform tasks more efficiently. For example, hammers, computers, and vehicles are all artifacts that augment our physical capabilities.

2. Language: This refers to the communication systems we use, including spoken and written words, mathematical symbols, and visual representations. Language allows us to share and record complex ideas, making it possible to collaborate and build on each other's knowledge over time.

3. Methodology: These are the methods, processes, and strategies we employ to organize our activities and solve problems. Methodologies can include scientific methods, management techniques, educational approaches, and more

"Douglas Engelbart identified four main ways in which human capabilities are augmented in his conceptual framework:\n\n1. Artifacts: These are the tools and physical instruments that humans create to manipulate the physical environment or to perform tasks more efficiently. For example, hammers, computers, and vehicles are all artifacts that augment our physical capabilities.\n\n2. Language: This refers to the communication systems we use, including spoken and written words, mathematical symbols, and visual representations. Language allows us to share and record complex ideas, making it possible to collaborate and build on each other's knowledge over time.\n\n3. Methodology: These are the methods, processes, and strategies we employ to organize our activities and solve problems. Methodologies can include scientific methods, management techniques, educational approaches, and more.\n\n4. Training: This encompasses the education and practice required to make effective use of artifacts, lan

In [27]:
quiz_questions = ask_pdf(pdf_qa, "Quiz me on the essay with 5 fundamental questions.")

quiz_questions

QUERY:  Quiz me on the essay with 5 fundamental questions.
ANSWER Certainly! Here are five fundamental questions about the essay "Augmenting Human Intellect: A Conceptual Framework" by Doug Engelbart:

1. What is the primary goal that Doug Engelbart envisions achieving through augmenting human intellect?
2. Describe one of the ways in which Engelbart suggests using computers to manipulate symbol structures differently from traditional pencil-and-paper techniques.
3. In the essay, what are the three questions Engelbart proposes that one might ask when looking at a statement in the middle of an argument structure?
4. How does Engelbart's concept of "linkages between different substructures" contribute to understanding complex statements?
5. What is the purpose of using "antecedent links" in the context of Engelbart's conceptual framework, and how does it help in understanding the basis upon which a statement was made?


'Certainly! Here are five fundamental questions about the essay "Augmenting Human Intellect: A Conceptual Framework" by Doug Engelbart:\n\n1. What is the primary goal that Doug Engelbart envisions achieving through augmenting human intellect?\n2. Describe one of the ways in which Engelbart suggests using computers to manipulate symbol structures differently from traditional pencil-and-paper techniques.\n3. In the essay, what are the three questions Engelbart proposes that one might ask when looking at a statement in the middle of an argument structure?\n4. How does Engelbart\'s concept of "linkages between different substructures" contribute to understanding complex statements?\n5. What is the purpose of using "antecedent links" in the context of Engelbart\'s conceptual framework, and how does it help in understanding the basis upon which a statement was made?'

In [22]:
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.0)

In [23]:
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

template = f"You take in text and spit out Python code doing what the user wants"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template("Return ONLY a PYTHON list containing the questions in this text: {questions}")

In [25]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

In [26]:
quiz_chain = chat_prompt | llm

In [85]:
quiz_chain.invoke({"questions": quiz_questions})

'questions = [\n    "What is the primary goal that Doug Engelbart envisions achieving through augmenting human intellect?",\n    "Describe one of the ways in which Engelbart suggests using computers to manipulate symbol structures differently from traditional pencil-and-paper techniques.",\n    "In the essay, what are the three questions Engelbart proposes that one might ask when looking at a statement in the middle of an argument structure?",\n    "How does Engelbart\'s concept of \'linkages between different substructures\' contribute to understanding complex statements?",\n    "What is the purpose of using \'antecedent links\' in the context of Engelbart\'s conceptual framework, and how does it help in understanding the basis upon which a statement was made?"\n]'

In [77]:
import re

def extract_python_code(markdown_string):
    pattern = r'```python\n(.*?)\n```'
    matches = re.findall(pattern, markdown_string, re.DOTALL)

    if matches:
        python_code = matches[0]
        return python_code
    else:
        return None

In [83]:
from langchain_core.runnables import RunnableLambda


quiz_chain = chat_prompt | llm | RunnableLambda(lambda x: x.content) | extract_python_code

Disclaimer: We haven't discussed runnable at length, but essentially they make up the core of the LCEL interface. 

`RunnableLambda` allows you to take in an output from part of the chain and pass it along after performing some transformation defined withint its lambda function.

In [87]:
questions_list = quiz_chain.invoke({"questions": quiz_questions})

In [89]:
exec(questions_list)

In [90]:
# the questions variable was created within the string inside the `questions_list` variable.
answers = []
for q in questions:
    answers.append(ask_pdf(pdf_qa,q))

QUERY:  What is the primary goal that Doug Engelbart envisions achieving through augmenting human intellect?
ANSWER The primary goal that Doug Engelbart envisions achieving through augmenting human intellect is to develop ways to increase the capability of a human to approach a complex problem situation, to gain comprehension to suit his particular needs, and to derive solutions to problems. Engelbart's focus was on enhancing human ability to solve problems, make decisions, and understand complex issues more effectively through the use of advanced tools, methodologies, and processes. This augmentation is not just about automating tasks but rather about empowering humans to do what was previously impossible or extremely difficult, thereby expanding both the intellectual and creative capacity of individuals and groups.
QUERY:  Describe one of the ways in which Engelbart suggests using computers to manipulate symbol structures differently from traditional pencil-and-paper techniques.
ANSW

In [91]:
evaluations = []

for q,a in zip(questions, answers):
    # Check for results
    evaluations.append(ask_pdf(pdf_qa,f"Is this: {a} the correct answer to this question: {q} according to the paper? Return ONLY '''YES''' or '''NO'''. Output:"))

evaluations

QUERY:  Is this: The primary goal that Doug Engelbart envisions achieving through augmenting human intellect is to develop ways to increase the capability of a human to approach a complex problem situation, to gain comprehension to suit his particular needs, and to derive solutions to problems. Engelbart's focus was on enhancing human ability to solve problems, make decisions, and understand complex issues more effectively through the use of advanced tools, methodologies, and processes. This augmentation is not just about automating tasks but rather about empowering humans to do what was previously impossible or extremely difficult, thereby expanding both the intellectual and creative capacity of individuals and groups. the correct answer to this question: What is the primary goal that Doug Engelbart envisions achieving through augmenting human intellect? according to the paper? Return ONLY '''YES''' or '''NO'''. Output:
ANSWER YES
QUERY:  Is this: Engelbart suggests that computers can

['YES', 'YES', 'YES', 'YES', 'YES']

In [93]:
scores = []

yes_count = evaluations.count('YES')
scores.append(yes_count)
print(scores)

[5]
